In [6]:
import numpy as np 
from HkVp_multilayer.ac import AC 
import numpy as np
from glob import glob 
import os
from obspy import read
import pandas as pd

In [7]:

maindir = "synthetic_data_S1.0"
ks = [1.9, 2.1, 2.3, 2.5, 2.7, 2.9, 3.1, 3.3, 3.5]
depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
excel_AC_para = pd.read_excel("AC_parameters10.xlsx")
paras = excel_AC_para.iloc[2:,:].values
noise_tag = "sn"

for k in ks:
    submaindir = f"{maindir}/Varyp_sediment_crust_mantle_synthetics_c35_ks{k}_dt0.01"
    for depth in depths:
        stationname = f"sediments_depth{depth}"
        datadir = f"{submaindir}/{stationname}"
        index = paras[:,0]==depth
        ##for P-wave
        f1s_P = paras[index,1][0]
        f2s_P = paras[index,2][0]
        f1c_P = paras[index,7][0]
        f2c_P = paras[index,8][0]
        ##for S-wave
        f1s_S = paras[index,4][0]
        f2s_S = paras[index,5][0]
        f1c_S = paras[index,10][0]
        f2c_S = paras[index,11][0]
        ## for width of spectral whetening
        dfs_P = paras[index,3][0]
        dfs_S = paras[index,6][0]
        dfc_P = paras[index,9][0]
        dfc_S = paras[index,12][0]

        print(f"parameter setting ({k} | {depth}):")
        print(f"f1s_P:{f1s_P} f2s_P:{f2s_P} dfs_P:{dfs_P}  f1s_S:{f1s_S} f2s_S:{f2s_S} dfs_S:{dfs_S}")
        print(f"f1c_P:{f1c_P} f2c_P:{f2c_P} dfc_P:{dfc_P}  f1c_S:{f1c_S} f2c_S:{f2c_S} dfc_S:{dfc_S}")

        ##for P-wave reflected at the bottom of sediments
        do_Ac = AC(datadir=datadir,suffix=f".Z{noise_tag}.SAC",tref=5.,pretime=-5,length=100,f1=f1s_P,f2=f2s_P,tcos=0.5,df=dfs_P)
        do_Ac.cal_ACs()
        stream_zacs = do_Ac.ACst

        ##for S-wave reflected at the bottom of sediments
        do_Ac = AC(datadir=datadir,suffix=f".R{noise_tag}.SAC",tref=5.,pretime=-5,length=150,f1=f1s_S,f2=f2s_S,tcos=1,df=dfs_S)
        do_Ac.cal_ACs()
        stream_racs = do_Ac.ACst

        ##for P-wave reflected at Moho
        do_Ac = AC(datadir=datadir,suffix=f".Z{noise_tag}.SAC",tref=5.,pretime=-5,length=100,f1=f1c_P,f2=f2c_P,tcos=5,df=dfc_P)
        do_Ac.cal_ACs()
        stream_zacm = do_Ac.ACst

        ##for S-wave reflected at Moho
        do_Ac = AC(datadir=datadir,suffix=f".R{noise_tag}.SAC",tref=5.,pretime=-5,length=150,f1=f1c_S,f2=f2c_S,tcos=5,df=dfc_S)
        do_Ac.cal_ACs()
        stream_racm = do_Ac.ACst

        ##read RF data
        stream_rf2 = read(f"{datadir}/ray*.R{noise_tag}.SAC_g2")
        stream_rf7 = read(f"{datadir}/ray*.R{noise_tag}.SAC_g5")

        delta = 0.01
        pretimes = [5,5,0,0,0,0]
        length = 100
        npts = int(length/delta)
        norm_type = [1,1,-1,-1,-1,-1]
        streams = [stream_rf7,stream_rf2,stream_zacs,stream_zacm,stream_racs,stream_racm]
        ndataset = len(streams)
        data_stack = np.zeros([ndataset,len(stream_rf7),int(length/delta)],dtype=np.float32)
        ray_params = []
        for i in range(ndataset):
            ntrace = len(streams[i])
            pretime = pretimes[i]
            ray_params_temp =  []
            data_st = []
            for j in range(ntrace):
                starttime = streams[i][j].stats.starttime
                streams[i][j].resample(1/delta)
                prenpts = int(pretime/delta)
                data = streams[i][j].data[prenpts:prenpts+npts]
                data = data/(data.max()-data.min())
                data_st.append(data)
                ray_params_temp.append(streams[i][j].stats.sac['user0'])
            data_ray_zip = zip(ray_params_temp,data_st)
            data_ray_zip_sorted = sorted(data_ray_zip,key=lambda x:x[0])
            ray_params_temp,data_st = zip(*data_ray_zip_sorted)
            for j,ray_param in enumerate(ray_params_temp):
                if i==0:
                    ray_params.append(ray_param)
                data_stack[i,j,:] = data_st[j]

        # Save the data_stack to a numpy file
        np.savez(f"{datadir}/data_stack52_10.npz", data_stack=data_stack, ray_params=ray_params)

parameter setting (1.9 | 1):
f1s_P:2 f2s_P:4 dfs_P:2  f1s_S:0.5 f2s_S:2.5 dfs_S:1
f1c_P:0.08 f2c_P:0.8 dfc_P:0.08  f1c_S:0.05 f2c_S:0.8 dfc_S:0.04
parameter setting (1.9 | 2):
f1s_P:0.5 f2s_P:2.2 dfs_P:0.5  f1s_S:0.3 f2s_S:2 dfs_S:0.5
f1c_P:0.08 f2c_P:0.8 dfc_P:0.08  f1c_S:0.05 f2c_S:0.8 dfc_S:0.04
parameter setting (1.9 | 3):
f1s_P:0.5 f2s_P:2.2 dfs_P:0.5  f1s_S:0.2 f2s_S:2.2 dfs_S:0.3
f1c_P:0.08 f2c_P:0.8 dfc_P:0.08  f1c_S:0.05 f2c_S:0.8 dfc_S:0.04
parameter setting (1.9 | 4):
f1s_P:0.5 f2s_P:2.2 dfs_P:0.5  f1s_S:0.2 f2s_S:2.2 dfs_S:0.3
f1c_P:0.08 f2c_P:0.8 dfc_P:0.08  f1c_S:0.05 f2c_S:0.8 dfc_S:0.04
parameter setting (1.9 | 5):
f1s_P:0.3 f2s_P:2.2 dfs_P:0.5  f1s_S:0.2 f2s_S:2.2 dfs_S:0.2
f1c_P:0.08 f2c_P:0.8 dfc_P:0.08  f1c_S:0.05 f2c_S:0.8 dfc_S:0.04
parameter setting (1.9 | 6):
f1s_P:0.3 f2s_P:2.2 dfs_P:0.5  f1s_S:0.2 f2s_S:2.2 dfs_S:0.2
f1c_P:0.08 f2c_P:0.8 dfc_P:0.08  f1c_S:0.05 f2c_S:0.8 dfc_S:0.04
parameter setting (1.9 | 7):
f1s_P:0.3 f2s_P:2.2 dfs_P:0.5  f1s_S:0.2 f2s_S:2.2 